In [6]:
import paramiko
import datetime
from datetime import datetime, timedelta
import os, zipfile
import re 
import os, glob
from collections import defaultdict
from datetime import datetime
import pandas as pd

In [71]:


# Open a transport
host = "ftp.coremetrics.com"
port = 998
transport = paramiko.Transport((host, port))
# Auth
password = "-rdGiuDi"
username = "50510000-export"
transport.connect(username = username, password = password)
# Go!
sftp = paramiko.SFTPClient.from_transport(transport)

# Get all relevant files in SFTP
files = []
for i in sftp.listdir():
    lstatout=str(sftp.lstat(i)).split()[0]
    if 'd' not in lstatout:
        if 'Trigger' not in i:
            files.append(i)

# Sort the files in order desc
def sort_DDF_data( l ): 
    convert = lambda text: int(text) if text.isdigit() else text 
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key)] 
    return sorted(l, key = alphanum_key, reverse=True)[0:8]


sorted_files = []
s = set(files) # make sure no duplicates
for x in sort_DDF_data(s):
    sorted_files.append(x)

#Dowwnload zipped files from SFTP
for file in sorted_files:
    filepath = file
    localpath = '/Users/john.ekedum@ibm.com/Downloads/DDF/' + file
    sftp.get(filepath, localpath)
    print(file + ' '+ 'downloaded')


# Unzip Zpped files
dir_name = '/Users/john.ekedum@ibm.com/Downloads/DDF/'
extension = ".zip"

os.chdir(dir_name) # change directory from working dir to dir with files

for item in os.listdir(dir_name):# loop through items in dir
    if item.endswith(extension): # check for ".zip" extension
        file_name = os.path.abspath(dir_name + item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall(dir_name + item[0:29]) # extract file to dir
        zip_ref.close() # close file
        os.remove(file_name) # delete zipped file

50510000_DigitalData_20180809.zip downloaded
50510000_DigitalData_20180808.zip downloaded
50510000_DigitalData_20180807.zip downloaded
50510000_DigitalData_20180806.zip downloaded
50510000_DigitalData_20180805.zip downloaded
50510000_DigitalData_20180804.zip downloaded
50510000_DigitalData_20180803.zip downloaded
50510000_DigitalData_20180802.zip downloaded


In [7]:
# loop through file names in directory and  stack them together according filenames 
dir_name = '/Users/john.ekedum@ibm.com/Downloads/DDF/'
list_of_dir=[]
for item in os.listdir(dir_name):
    list_of_dir.append(r'/Users/john.ekedum@ibm.com/Downloads/DDF/' + item)
    
dirs = list_of_dir
unique_files = defaultdict(list)


for file_pattrn in ['Element.txt', 'Geography.txt', 'PageView.txt', 'ConversionEvent.txt', 'TechnicalProperties.txt', 'Registration.txt']:
    for d in dirs:
        for i in glob.iglob(os.path.join(d, file_pattrn)):
            unique_files[os.path.basename(i)].append(i)
            
destination = '/Users/john.ekedum@ibm.com/Downloads/DDF'
for unique_filename, copies in unique_files.items():
    with open(os.path.join(destination, unique_filename), 'w') as f:
        for copy in copies:
            with open(copy, 'r') as cp:
                for line in cp:
                    f.write(line)

In [9]:
path = '/Users/john.ekedum@ibm.com/Downloads/DDF/'

In [166]:
#Page views done
df_pageview = pd.read_csv('/Users/john.ekedum@ibm.com/Downloads/DDF/PageView.txt')
mask = df_pageview.TIMESTAMP  == 'TIMESTAMP'
df_pageview = df_pageview.loc[~mask,::]
df_pageview.TIMESTAMP = pd.to_datetime(df_pageview.TIMESTAMP)
cols_drop = df_pageview.columns[21:63]
df_pageview.drop(cols_drop, axis=1, inplace=True)
df_pageview.columns = df_pageview.columns.str.title().str.replace(' ','_')
df_pageview.to_csv(os.path.join(path,r'PageView.csv'),index=False)

/Users/john.ekedum@ibm.com/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,9,13,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [155]:
df_pageview.Page_View_Attribute_19.unique()


array([nan, '63E1BEB9-BC48-4B46-91EF-A4D4E5C697CB',
       'CA79E2F2-F17B-4BD6-B3B8-A4F5C69A3BB9', ...,
       'D106851B-A2BA-4773-8F93-A09A3302EE6D',
       'FA1E99C9-D20D-4232-969A-96094E3C2E4D',
       'B3421E14-589E-4877-89DC-B50106171E6A'], dtype=object)

In [179]:
# Element done
df_element = pd.read_csv('/Users/john.ekedum@ibm.com/Downloads/DDF/Element.txt')
mask = df_element.TIMESTAMP  == 'TIMESTAMP'
df_element = df_element.loc[~mask,::]
df_element.TIMESTAMP = pd.to_datetime(df_element.TIMESTAMP)
cols_drop = df_element.columns[17:56]
df_element.drop(cols_drop, axis=1, inplace=True)
df_element.columns = df_element.columns.str.title().str.replace(' ','_')
df_element.to_csv(os.path.join(path,r'Element.csv'), index=False)

Index(['Session_Id', 'Cookie_Id', 'Timestamp', 'Element_Name',
       'Element_Category', 'Site_Id', 'Cid', 'Client_Name', 'Username',
       'Data_Center', 'Locale', 'Country', 'Subscription_Type', 'Environment',
       'Current_Page_Url', 'Referring_Page_Url', 'Element_Attribute_11'],
      dtype='object')

In [90]:
#Conversion done
df_conv = pd.read_csv('/Users/john.ekedum@ibm.com/Downloads/DDF/ConversionEvent.txt')
mask = df_conv.SESSION_ID == 'SESSION_ID'
df_conv = df_conv.loc[~mask,::]
df_conv = df_conv.iloc[::,0:15]
df_conv.columns = df_conv.columns.str.title().str.replace(' ','_')
df_conv.to_csv(os.path.join(path,r'ConversionEvent.csv'), index=False)


/Users/john.ekedum@ibm.com/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,5,6,8,10,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [91]:

df_conv.head()

,Session_Id,Cookie_Id,Timestamp,Event_Name,Event_Category,Event_Action_Type,Event_Points,Site_Id,Client_Id,User_Id,Remember_Id_&_User,Locale,Data_Center,Company_Name,Email
0,1898463527422067987,25759794830214857936680,2018-07-24 07:47:31,WELCOME:REPORT LOAD:TOP PAGES,REPORT LOAD,1,0,WELCOME,90301777,SSHRAGO,True,EN_US,US,NaN,NaN
1,1898463527422067987,25759794830214857936680,2018-07-24 07:46:31,WELCOME:REPORT LOAD:MARKETING PROGRAMS | BY VE...,REPORT LOAD,2,0,WELCOME,90301777,SSHRAGO,True,EN_US,US,2485,NaN
2,1898463527422067987,25759794830214857936680,2018-07-24 07:47:10,WELCOME:REPORT LOAD:TOP PAGES,REPORT LOAD,1,0,WELCOME,90301777,SSHRAGO,True,EN_US,US,NaN,NaN
3,1898463527422067987,25759794830214857936680,2018-07-24 07:47:15,WELCOME:REPORT LOAD:TOP PAGES,REPORT LOAD,2,0,WELCOME,90301777,SSHRAGO,True,EN_US,US,5581,NaN
4,1898463527422067987,25759794830214857936680,2018-07-24 07:46:28,WELCOME:REPORT LOAD:MARKETING PROGRAMS | BY VE...,REPORT LOAD,1,0,WELCOME,90301777,SSHRAGO,True,EN_US,US,NaN,NaN


In [94]:
#technical properties

650659

In [316]:
df_tech = pd.read_csv('/Users/john.ekedum@ibm.com/Downloads/DDF/TechnicalProperties.txt')
mask = df_tech.SESSION_ID == 'SESSION_ID'
df_tech = df_tech.loc[~mask,::]
df_tech.columns = df_tech.columns.str.title().str.replace(' ','_')
df_tech.to_csv(os.path.join(path,r'TechnicalProperties.csv'), index=False)

In [ ]:
#Registration

In [213]:
df_reg = pd.read_csv('/Users/john.ekedum@ibm.com/Downloads/DDF/Registration.txt')
mask = df_reg.SESSION_ID == 'SESSION_ID'
df_reg = df_reg.loc[~mask,::]
cols_drop = df_reg.columns[21:63]
df_reg.drop(cols_drop, axis=1, inplace=True)
df_reg.columns = df_reg.columns.str.title().str.replace(' ','_')
df_reg.to_csv(os.path.join(path,r'Registration.csv'), index=False)

/Users/john.ekedum@ibm.com/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,3,4,6,7,8,9,10,12,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# Geography

In [ ]:
my_file = '/Users/john.ekedum@ibm.com/Downloads/DDF/50510000_DigitalData_20180702/Geography.txt'

In [247]:
import numpy as np
line2 = np.linspace(801, 809, 9, dtype=int).tolist()
line = np.linspace(703, 731, 29 , dtype=int).tolist()
for list in line2:
    line.append(list)
my_file =[]
for li in line:
    my_file.append('/Users/john.ekedum@ibm.com/Downloads/DDF/50510000_DigitalData_20180' + str(li) + '/Geography.txt')
    

In [257]:
for names in my_file:
    print(names[62:70])[1:2]


In [251]:
import csv
for names in my_file:
    with open(names,'r') as csvinput:
        with open('/Users/john.ekedum@ibm.com/Downloads/GEO/' + names[42 :70] + '_geo'+ '.csv', 'w') as csvoutput:
            writer = csv.writer(csvoutput, lineterminator='\n')
            reader = csv.reader(csvinput)
            all = []
            row = next(reader)
            row.append('Date')
            all.append(row)
            for row in reader:
                row.append(names[62:70])
                all.append(row)
        
            writer.writerows(all)

In [252]:
dir_name2 = '/Users/john.ekedum@ibm.com/Downloads/GEO'
file_geo = os.listdir(dir_name2)
file_geo.remove('.DS_Store')

['0510000_DigitalData_20180726_geo.csv',
 '0510000_DigitalData_20180719_geo.csv',
 '0510000_DigitalData_20180709_geo.csv',
 '0510000_DigitalData_20180710_geo.csv',
 '0510000_DigitalData_20180807_geo.csv',
 '0510000_DigitalData_20180711_geo.csv',
 '0510000_DigitalData_20180806_geo.csv',
 '0510000_DigitalData_20180727_geo.csv',
 '0510000_DigitalData_20180708_geo.csv',
 '0510000_DigitalData_20180718_geo.csv',
 '0510000_DigitalData_20180804_geo.csv',
 '0510000_DigitalData_20180703_geo.csv',
 '0510000_DigitalData_20180713_geo.csv',
 '0510000_DigitalData_20180725_geo.csv',
 '0510000_DigitalData_20180724_geo.csv',
 '0510000_DigitalData_20180805_geo.csv',
 '0510000_DigitalData_20180712_geo.csv',
 '0510000_DigitalData_20180707_geo.csv',
 '0510000_DigitalData_20180717_geo.csv',
 '0510000_DigitalData_20180728_geo.csv',
 '0510000_DigitalData_20180809_geo.csv',
 '0510000_DigitalData_20180721_geo.csv',
 '0510000_DigitalData_20180731_geo.csv',
 '0510000_DigitalData_20180808_geo.csv',
 '0510000_Digita

In [262]:
fout=open("/Users/john.ekedum@ibm.com/Downloads/GEO/final.csv","a")

# first file:
for lines in open("/Users/john.ekedum@ibm.com/Downloads/GEO/0510000_DigitalData_20180703_geo.csv"):
    fout.write(lines)
# now the rest:    
for num in line[1::]:
    f = open("/Users/john.ekedum@ibm.com/Downloads/GEO/0510000_DigitalData_20180"+ str(num) + "_geo.csv")
    f.__next__()# skip the header
    for lines in f:
         fout.write(lines)
    f.close() # not really needed
fout.close()

In [317]:
df_geo = pd.read_csv("/Users/john.ekedum@ibm.com/Downloads/GEO/final.csv")
mask = df_geo.SESSION_ID == 'SESSION_ID'
df_geo = df_geo.loc[~mask,::]
df_geo.columns = df_geo.columns.str.title().str.replace(' ','_')
df_geo.Date = df_geo.Date.astype('str')
df_geo.Date.str.slice(0,4) + "-" + df_geo.Date.str.slice(4,6) + "-" + df_geo.Date.str.slice(6,8)
df_geo.Date = pd.to_datetime(df_geo.Date)
df_geo.to_csv(os.path.join(path,r'Geography.csv'), index=False)

/Users/john.ekedum@ibm.com/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [319]:
df_geo.shape

(478245, 9)

In [ ]:
schedule.every().schedule.every().monday.at("14:00").do(job)
schedule.every().tuesday.at("14:00").do(job)
schedule.every().wednesday.at("14:00").do(job)
schedule.every().thursday.at("14:00").do(job)
schedule.every().friday.at("14:00").do(job)

while True:
    schedule.run_pending()
    time.sleep(1)



In [83]:
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected!")

except:
    print ("Unable to connect to database")


Unable to connect to database
